In [ ]:
import glob, os
import cv2
from PIL import Image
import numpy as np
import torch

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
root = os.path.join('/content/drive/MyDrive/Data')
images_directory = os.path.join(root, 'image_dataset')
masks_directory = os.path.join(root, 'mask_dataset')

images_filenames = list(sorted(os.listdir(images_directory)))
print(images_filenames)


['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '3.jpg', '30.jpg', '31.jpg', '97.jpg', 'GrassD_1.jpg', 'GrassD_100.jpg', 'GrassD_1048.jpg', 'GrassD_107.jpg', 'GrassD_109.jpg', 'GrassD_110.jpg', 'GrassD_113.jpg', 'GrassD_114.jpg', 'GrassD_116.jpg', 'GrassD_118.jpg', 'GrassD_120.jpg', 'GrassD_127.jpg', 'GrassD_13.jpg', 'GrassD_1437.jpg', 'GrassD_1439.jpg', 'GrassD_1467.jpg', 'GrassD_1468.jpg', 'GrassD_1470.jpg', 'GrassD_1474.jpg', 'GrassD_1475.jpg', 'GrassD_1478.jpg', 'GrassD_148.jpg', 'GrassD_1486.jpg', 'GrassD_1500.jpg', 'GrassD_1501.jpg', 'GrassD_1543.jpg', 'GrassD_1567.jpg', 'GrassD_1578.jpg', 'GrassD_1579.jpg', 'GrassD_1582.jpg', 'GrassD_1584.jpg', 'GrassD_159.jpg', 'GrassD_1613.jpg', 'GrassD_1614.jpg', 'GrassD_1618.jpg', 'GrassD_162.jpg', 'GrassD_1620.jpg', 'GrassD_1621.jpg', 'GrassD_1640.jpg', 'GrassD_1641.jpg', 'GrassD_1652.jpg', '

In [ ]:
img_names = glob.glob('/content/drive/MyDrive/Data/image_dataset/*.jpg')
msk_names = glob.glob('/content/drive/MyDrive/Data/mask_dataset/*.png')
print(len(img_names), len(msk_names))

361 361


In [ ]:
class_names = ['background', 'grass', 'dog', 'tree', 'sky', 'concrete/walkway']

class_rgb_values = [
                    [0,0,0], [0,128,0],[128,128,128], [0,0,128], [128,0,128], [128,0,0]
                    ]

In [ ]:
select_classes = ['background', 'grass']

#class_rgb_values = [[0,0,0], [0,128,0],[128,128,128], [0,0,128], [128,0,128], [128,0,0]]

select_class_indices = [class_names.index(cls.lower()) for cls in select_classes]
select_class_rgb_values =  np.array(class_rgb_values)[select_class_indices]

In [ ]:
def one_hot_encode(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes
    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map

In [ ]:
# Perform reverse one-hot-encoding on labels / preds
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.
    # Arguments
        image: The one-hot format image 
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified 
        class key.
    """
    x = np.argmax(image, axis = -1)
    return x

In [ ]:
# Perform colour coding on the reverse-one-hot outputs
def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.
    # Arguments
        image: single channel array where each value represents the class key.
        label_values

    # Returns
        Colour coded image for segmentation visualization
    """
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x

In [ ]:
class BackgroundDataset(torch.utils.data.Dataset):

    
    def __init__(
            self, 
            images_filenames,
            images_directory,
            masks_directory,
            class_rgb_values=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.masks_directory = masks_directory
        
        
        self.class_rgb_values = class_rgb_values
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, idx):
        
        image_filename = self.images_filenames[idx]
        image = cv2.imread(os.path.join(self.images_directory, image_filename))
        image = cv2.resize(image, (256,256))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(
            os.path.join(self.masks_directory, image_filename.replace(".jpg", ".png")), cv2.COLOR_BGR2RGB,
        )
        mask = cv2.resize(mask, (256,256))
        
        # one-hot-encode the mask
        mask = one_hot_encode(mask, self.class_rgb_values).astype('float')
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.images_filenames)

In [ ]:


allMask = BackgroundDataset(images_filenames, images_directory, masks_directory, class_rgb_values=select_class_rgb_values)
# random_idx = random.randint(0, len(dataset)-1)
# image, mask = allMask
names = []
for mask in msk_names:
    name = mask.rsplit('/', 1)[1]
    # name = mask[-4:]
    names.append(name)
names.sort()
c = 0
for i, (img, msk) in enumerate(allMask):
  #name = 'img'+str(c)+'.png'
  ground_truth_mask = colour_code_segmentation(reverse_one_hot(msk), select_class_rgb_values)
  pil = im = Image.fromarray((ground_truth_mask * 255).astype(np.uint8))
  pil.save('/content/drive/MyDrive/Data/binary_mask_dataset/' + names[i])
  c+=1

In [ ]:
root = os.path.join('/content/drive/MyDrive/Data')
images_directory2 = os.path.join(root, 'image_dataset')
masks_directory2 = os.path.join(root, 'binary_mask_dataset')

In [ ]:
img_names = glob.glob('/content/drive/MyDrive/Data/image_dataset/*.jpg')
msk_names = glob.glob('/content/drive/MyDrive/Data/binary_mask_dataset/*.png')


In [ ]:
img_names.sort()
msk_names.sort()

In [ ]:
img = '/content/drive/MyDrive/Data/image_dataset/*.jpg'
files = [filename for filename in os.listdir(img)]


In [ ]:
src3 = r'/content/drive/MyDrive/Data/mask/'
dest = '/content/drive/MyDrive/Data/mask2/'
files = [filename for filename in os.listdir(src3)]
# print(files)
# # files.sort()
# j = 1
# for i in files:
#     img = Image.open(src3 + i).convert('RGB')
#     #name = "GrassD_" + str(j) + ".jpg"
#     img.save(dest + str(j) + ".jpg")
#     j += 1

['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg', '51.jpg', '52.jpg', '53.jpg', '54.jpg', '55.jpg', '56.jpg', '57.jpg', '58.jpg', '59.jpg', '60.jpg', '61.jpg', '62.jpg', '63.jpg', '64.jpg', '65.jpg', '66.jpg', '67.jpg', '68.jpg', '69.jpg', '70.jpg', '71.jpg', '72.jpg', '73.jpg', '74.jpg', '75.jpg', '76.jpg', '77.jpg', '78.jpg', '79.jpg', '80.jpg', '81.jpg', '82.jpg', '83.jpg', '84.jpg', '85.jpg', '86.jpg', '87.jpg', '88.jpg', '89.jpg', '90.jpg', '91.jpg', '92.jpg', '93.jpg', '94.jpg', '95.jpg', '96.jpg', '97.jpg', '98.jpg', '99.jpg', '100.jpg', '101.jp

In [ ]:

# print(img_names[0])
# print(msk_names[0])
# list = [line.rsplit('/', 1)[1] for line in img_names]
# print(list)
textfile = open('/content/drive/MyDrive/Data/trainval.txt', "w")
for e in range(1, 362):
  textfile.write(str(e) + "\n")

textfile.close()



In [ ]:
textfile2 = open('/content/drive/MyDrive/Data/train.txt', "w")
for e in range(1,337):
  textfile2.write(str(e) + "\n")
textfile2.close()

In [ ]:
textfile3 = open('/content/drive/MyDrive/Data/val.txt', "w")
for e in range(337, 362):
  textfile3.write(str(e) + "\n")
textfile3.close()

In [4]:
a = [[[5,10,15,30,25],[20,30,65,70,90],[7,80,95,20,30]]
[[3,0,5,0,45],
  [12,-2,6,7,90],
  [18,-9,95,120,30]]
 [[17,13,25,30,15],
  [23,36,9,7,80],
  [1,-7,-5,22,3]]]

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<ipython-input-4-586f1a4850e8>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  a = [[[5,10,15,30,25],[20,30,65,70,90],[7,80,95,20,30]]


TypeError: ignored

In [9]:
a = [4,5]
b = [4,5]

import numpy as np
c = np.dot(a, b)
c

41